In [ ]:
# repo_dir = "Repos"   # Set this to be where your github repos are located.
# %load_ext autoreload
# %autoreload 2

# # Update the load path so python can find modules for the model
# import sys
# from pathlib import Path
# sys.path.insert(0, str(Path.home() / repo_dir / "eye-ai-ml"))
# sys.path.insert(0, str(Path.home() / repo_dir / "deriva-ml"))

In [1]:
# Prerequisites
import json
import os
from eye_ai.eye_ai import EyeAI

import pandas as pd
from pathlib import Path, PurePath
import logging

from deriva_ml import DatasetBag, Workflow, ExecutionConfiguration
from deriva_ml import MLVocab as vc
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

2025-02-12 13:17:11.770305: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-12 13:17:11.770354: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-12 13:17:11.771196: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-12 13:17:11.777722: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-12 13:17:12.532140: W tensorflow/compiler/tf2

In [2]:
# Login
from deriva.core.utils.globus_auth_utils import GlobusNativeLogin
# host = 'www.eye-ai.org'
host = 'dev.eye-ai.org'
catalog_id = "eye-ai"

gnl = GlobusNativeLogin(host=host)
if gnl.is_logged_in([host]):
    print("You are already logged in.")
else:
    gnl.login([host], no_local_server=True, no_browser=True, refresh_tokens=True, update_bdbag_keychain=True)
    print("Login Successful")

2025-02-12 13:17:13,075 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2025-02-12 13:17:13,076 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>


You are already logged in.


In [3]:
cache_dir = '/data'
working_dir = '/data'
EA = EyeAI(hostname = host, catalog_id = catalog_id, cache_dir= cache_dir, working_dir=working_dir)

2025-02-12 13:17:13,084 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2025-02-12 13:17:13,085 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>


# 1. Configuration

In [4]:
# RID of source dataset, if any.
source_dataset = '2-7KA2'

EA.add_term(vc.workflow_type, "Test Workflow", description="A test Workflow for new DM")
# Workflow instance
test_workflow = Workflow(
    name="LAC data template",
    url="https://github.com/informatics-isi-edu/eye-ai-exec/blob/main/notebooks/templates/template_lac.ipynb",
    workflow_type="Test Workflow"
)

# Configuration instance.
config = ExecutionConfiguration(
    datasets=[{'rid':source_dataset, 'materialize':True}],
    # Materialize set to False if you only need the metadata from the bag, and not the assets.
    assets=['2-4JR6'],
    workflow=test_workflow,
    description="Template instance of a feature creation workflow")

# Initialize execution
execution = EA.create_execution(config)


2025-02-12 13:17:21,980 - INFO - File [/data/lizhiwei/EyeAI_working/5-SYBW/asset/optic_disk_crop_model.hdf5] transfer successful. 76.75 MB transferred at 11.70 MB/second. Elapsed time: 0:00:06.559468.


In [5]:
print(execution)

caching_dir: /data
working_dir: /data/lizhiwei/EyeAI_working
execution_rid: 5-SYBW
workflow_rid: 5-SGC2
asset_paths: [PosixPath('/data/lizhiwei/EyeAI_working/5-SYBW/asset/optic_disk_crop_model.hdf5')]
configuration: datasets=[DatasetSpec(rid='2-7KA2', materialize=True, version=None)] assets=['2-4JR6'] workflow=Workflow(name='LAC data template', url='https://github.com/informatics-isi-edu/eye-ai-exec/blob/main/notebooks/templates/template_lac.ipynb', workflow_type='Test Workflow', version=None, description=None) description='Template instance of a feature creation workflow'


# 2. Get access to the data

In [6]:
# Find all the dataset bags get downloaded. (Here in this example, we only have one dataset)
print(execution.datasets)

# retrieve each dataset bag
ds_bag = execution.datasets[0]
ds_bag

[<deriva_ml.DatasetBag object 2-7KA2 at 0x7fb31bd07c10>]


<deriva_ml.DatasetBag object 2-7KA2 at 0x7fb31bd07c10>

## 2.1 Check all the tables in the dataset bags

In [7]:
ds_bag.list_tables()

['deriva-ml:Dataset',
 'deriva-ml:Dataset_Dataset',
 'deriva-ml:Dataset_Dataset_Type',
 'deriva-ml:Dataset_Execution',
 'deriva-ml:Dataset_Type',
 'deriva-ml:Dataset_Version',
 'deriva-ml:Execution',
 'deriva-ml:Execution_Asset',
 'deriva-ml:Execution_Asset_Execution',
 'deriva-ml:Execution_Asset_Type',
 'deriva-ml:Execution_Metadata',
 'deriva-ml:Execution_Metadata_Execution',
 'deriva-ml:Execution_Metadata_Type',
 'deriva-ml:Feature_Name',
 'deriva-ml:Workflow',
 'deriva-ml:Workflow_Type',
 'eye-ai:Annotation',
 'eye-ai:Annotation_Function',
 'eye-ai:Annotation_Type',
 'eye-ai:Clinical_Records',
 'eye-ai:Clinical_Records_ICD10_Eye',
 'eye-ai:Clinical_Records_Observation',
 'eye-ai:Condition_Label',
 'eye-ai:Diagnosis_Image',
 'eye-ai:Diagnosis_Observation',
 'eye-ai:Diagnosis_Status',
 'eye-ai:Diagnosis_Subject',
 'eye-ai:Diagnosis_Tag',
 'eye-ai:Execution_Image_Fundus_Angle',
 'eye-ai:Execution_Image_Fundus_Laterality',
 'eye-ai:Fundus_Bounding_Box',
 'eye-ai:ICD10_Eye',
 'eye-ai:Im

## 2.2 Load data in a table to a dataframe 
Use method `ds_bag.get_table_as_dataframe(<table_name>)`\
Examples:

In [18]:
subject_table = ds_bag.get_table_as_dataframe('Subject')
observation_table = ds_bag.get_table_as_dataframe('Observation')
image_table = ds_bag.get_table_as_dataframe('Image')

In [9]:
subject_table

,RID,RCT,RMT,RCB,RMB,Subject_ID,Subject_Gender,Subject_Ethnicity
0,6RDP,2023-04-24 23:00:34.133372+00,2024-10-18 18:01:15.622589+00,https://auth.globus.org/2a32b19e-2728-4708-827...,https://auth.globus.org/3769492a-b197-4063-952...,LAC_696040,F,Other
1,6RE0,2023-04-24 23:00:34.133372+00,2024-10-18 18:01:15.622589+00,https://auth.globus.org/2a32b19e-2728-4708-827...,https://auth.globus.org/3769492a-b197-4063-952...,LAC_900811,M,Asian
2,6REA,2023-04-24 23:00:34.133372+00,2024-10-18 18:01:15.622589+00,https://auth.globus.org/2a32b19e-2728-4708-827...,https://auth.globus.org/3769492a-b197-4063-952...,LAC_963487,M,African Descent
3,6RF0,2023-04-24 23:00:34.133372+00,2024-10-18 18:01:15.622589+00,https://auth.globus.org/2a32b19e-2728-4708-827...,https://auth.globus.org/3769492a-b197-4063-952...,LAC_428549,M,Latin American
4,6RGY,2023-04-24 23:00:34.133372+00,2024-10-18 18:01:15.622589+00,https://auth.globus.org/2a32b19e-2728-4708-827...,https://auth.globus.org/3769492a-b197-4063-952...,LAC_612937,F,Latin American
5,6RK2,2023-04-24 23:00:34.133372+00,2024-10-18 18:01:15.622589+00,https://auth.globus.org/2a32b19e-2728-4708-827...,https://auth.globus.org/3769492a-b197-4063-952...,LAC_894955,M,Latin American
6,6RMA,2023-04-24 23:00:34.133372+00,2024-10-18 18:01:15.622589+00,https://auth.globus.org/2a32b19e-2728-4708-827...,https://auth.globus.org/3769492a-b197-4063-952...,LAC_399033,F,Latin American
7,6RMP,2023-04-24 23:00:34.133372+00,2024-10-18 18:01:15.622589+00,https://auth.globus.org/2a32b19e-2728-4708-827...,https://auth.globus.org/3769492a-b197-4063-952...,LAC_966607,F,African Descent
8,6RN4,2023-04-24 23:00:34.133372+00,2024-10-18 18:01:15.622589+00,https://auth.globus.org/2a32b19e-2728-4708-827...,https://auth.globus.org/3769492a-b197-4063-952...,LAC_852885,F,Latin American
9,6RNA,2023-04-24 23:00:34.133372+00,2024-10-18 18:01:15.622589+00,https://auth.globus.org/2a32b19e-2728-4708-827...,https://auth.globus.org/3769492a-b197-4063-952...,LAC_910138,M,ethnicity not specified


## 2.3 Handling nested dataset
We have already downloaded the dataset **2-7KA2**, which contains **15** subjects. We call it a full set.\
The dataset **2-7KA2** has a **subset 2-7K8W** which contains **10** subjects from the full set. \
The following scripts show how to get access to the data in a nested dataset.


In [13]:
subset_2_7K8W_members = EA.list_dataset_members('2-7K8W') # Only the 10 subjects from the subset will be listed here 
subset_2_7K8W_members

{'Image': [],
 'Subject': [{'RID': '6RDP',
   'RCT': '2023-04-24T23:00:34.133372+00:00',
   'RMT': '2024-10-18T18:01:15.622589+00:00',
   'RCB': 'https://auth.globus.org/2a32b19e-2728-4708-827d-25398c1d83b1',
   'RMB': 'https://auth.globus.org/3769492a-b197-4063-952e-2500351dbcdd',
   'Subject_ID': 'LAC_696040',
   'Subject_Gender': 'F',
   'Subject_Ethnicity': 'Other'},
  {'RID': '6RE0',
   'RCT': '2023-04-24T23:00:34.133372+00:00',
   'RMT': '2024-10-18T18:01:15.622589+00:00',
   'RCB': 'https://auth.globus.org/2a32b19e-2728-4708-827d-25398c1d83b1',
   'RMB': 'https://auth.globus.org/3769492a-b197-4063-952e-2500351dbcdd',
   'Subject_ID': 'LAC_900811',
   'Subject_Gender': 'M',
   'Subject_Ethnicity': 'Asian'},
  {'RID': '6REA',
   'RCT': '2023-04-24T23:00:34.133372+00:00',
   'RMT': '2024-10-18T18:01:15.622589+00:00',
   'RCB': 'https://auth.globus.org/2a32b19e-2728-4708-827d-25398c1d83b1',
   'RMB': 'https://auth.globus.org/3769492a-b197-4063-952e-2500351dbcdd',
   'Subject_ID': 'L

In [20]:
# Create a dataframe of all the subjects in the subset.
subject_2_7K8W = pd.DataFrame(subset_2_7K8W_members['Subject'])
subject_2_7K8W

# Use join to get the Observation/Image/Diagnosis that only belong to the subset
observation_2_7K8W = pd.merge(subject_2_7K8W, observation_table, left_on='RID', right_on='Subject')
image_2_7K8W = pd.merge(observation_2_7K8W, image_table, left_on='RID_y', right_on='Observation')
image_2_7K8W

,RID_x,RCT_x,RMT_x,RCB_x,RMB_x,Subject_ID,Subject_Gender,Subject_Ethnicity,RID_y,RCT_y,...,Description,Length,MD5,Observation,Image_ID,Source_Image,Process,Image_Side,Image_Angle,Image_Tag
0,6RDP,2023-04-24T23:00:34.133372+00:00,2024-10-18T18:01:15.622589+00:00,https://auth.globus.org/2a32b19e-2728-4708-827...,https://auth.globus.org/3769492a-b197-4063-952...,LAC_696040,F,Other,76M2,2023-05-02 00:48:38.860018+00,...,,607528,23f8d0c98f6b13930b800630f04a7732,76M2,10744912,,,Right,3,Source
1,6RDP,2023-04-24T23:00:34.133372+00:00,2024-10-18T18:01:15.622589+00:00,https://auth.globus.org/2a32b19e-2728-4708-827...,https://auth.globus.org/3769492a-b197-4063-952...,LAC_696040,F,Other,76M2,2023-05-02 00:48:38.860018+00,...,,635932,3f2b4ecbe2a4e2011779fa126a2bbeb1,76M2,10744913,,,Left,1,Source
2,6RDP,2023-04-24T23:00:34.133372+00:00,2024-10-18T18:01:15.622589+00:00,https://auth.globus.org/2a32b19e-2728-4708-827...,https://auth.globus.org/3769492a-b197-4063-952...,LAC_696040,F,Other,76M2,2023-05-02 00:48:38.860018+00,...,,667204,43fae09608ec194b475e5d44f4c7d23b,76M2,10744914,,,Right,1,Source
3,6RDP,2023-04-24T23:00:34.133372+00:00,2024-10-18T18:01:15.622589+00:00,https://auth.globus.org/2a32b19e-2728-4708-827...,https://auth.globus.org/3769492a-b197-4063-952...,LAC_696040,F,Other,76M2,2023-05-02 00:48:38.860018+00,...,,666612,ad23ac6270cff974ee42e18c73d75ba1,76M2,10744915,,,Left,2,Source
4,6RDP,2023-04-24T23:00:34.133372+00:00,2024-10-18T18:01:15.622589+00:00,https://auth.globus.org/2a32b19e-2728-4708-827...,https://auth.globus.org/3769492a-b197-4063-952...,LAC_696040,F,Other,76M2,2023-05-02 00:48:38.860018+00,...,,609759,542841ee62a8298b69159377ca43e3e6,76M2,10744916,,,Left,3,Source
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,6RNA,2023-04-24T23:00:34.133372+00:00,2024-10-18T18:01:15.622589+00:00,https://auth.globus.org/2a32b19e-2728-4708-827...,https://auth.globus.org/3769492a-b197-4063-952...,LAC_910138,M,ethnicity not specified,76WM,2023-05-02 00:48:38.860018+00,...,,162215,39198754f8a7877e108caf7cdd6707db,76WM,13369011,,,Left,2,Source
62,6RNA,2023-04-24T23:00:34.133372+00:00,2024-10-18T18:01:15.622589+00:00,https://auth.globus.org/2a32b19e-2728-4708-827...,https://auth.globus.org/3769492a-b197-4063-952...,LAC_910138,M,ethnicity not specified,76WM,2023-05-02 00:48:38.860018+00,...,,163959,6223b40266f63411c398153464d816e3,76WM,13369012,,,Right,2,Source
63,6RNA,2023-04-24T23:00:34.133372+00:00,2024-10-18T18:01:15.622589+00:00,https://auth.globus.org/2a32b19e-2728-4708-827...,https://auth.globus.org/3769492a-b197-4063-952...,LAC_910138,M,ethnicity not specified,76WM,2023-05-02 00:48:38.860018+00,...,,166191,ca770e6f7339277449bdf33b445d4427,76WM,13369013,,,Left,1,Source
64,6RNA,2023-04-24T23:00:34.133372+00:00,2024-10-18T18:01:15.622589+00:00,https://auth.globus.org/2a32b19e-2728-4708-827...,https://auth.globus.org/3769492a-b197-4063-952...,LAC_910138,M,ethnicity not specified,76WM,2023-05-02 00:48:38.860018+00,...,,165604,f2e32d6cd9b0ea5cd7e44f9fb5e2a04d,76WM,13369016,,,Right,1,Source


# 3. Get access to the downloaded assets

In [16]:
print(execution.asset_paths)

[PosixPath('/data/lizhiwei/EyeAI_working/5-SYBW/asset/optic_disk_crop_model.hdf5')]


# 4. Methods for LAC data

### Filter on angle 2 image

In [ ]:
angle2_image = EA.filter_angle_2(ds_bag)

### Diagnosis manipulation

In [10]:
init_diag = EA.image_tall(ds_bag, 'Initial Diagnosis')
grader_diag = EA.image_tall(ds_bag, 'AI_glaucomasuspect_test')
long, wide = EA.reshape_table([init_diag, grader_diag], 'Diagnosis_Image')

eye-ai:Subject
eye-ai:Observation
eye-ai:Image
eye-ai:Image_Diagnosis
eye-ai:Subject
eye-ai:Observation
eye-ai:Image
eye-ai:Image_Diagnosis


In [19]:
init_diag

,Subject_RID,Image_RID,Diagnosis_RID,Full_Name,Image_Side,Diagnosis_Image,Cup_Disk_Ratio,Image_Quality
78,6RDP,9ZYM,5-NKN0,Initial Diagnosis,Left,Suspected Glaucoma,,Good
96,6RDP,9ZYR,5-NKN4,Initial Diagnosis,Right,Suspected Glaucoma,,Good
116,6RE0,AAZ6,5-NYNJ,Initial Diagnosis,Left,Suspected Glaucoma,,Good
135,6RE0,AAZC,5-NYNR,Initial Diagnosis,Right,Suspected Glaucoma,,Good
157,6REA,AF76,5-P2X4,Initial Diagnosis,Right,Suspected Glaucoma,,Good
175,6REA,AF7A,5-P2X8,Initial Diagnosis,Left,Suspected Glaucoma,,Good
195,6RF0,9F04,5-N2PJ,Initial Diagnosis,Right,Suspected Glaucoma,,Good
212,6RF0,9F06,5-N2PM,Initial Diagnosis,Left,Suspected Glaucoma,,Good
230,6RGY,8DWG,5-M1VR,Initial Diagnosis,Left,No Glaucoma,,Good
247,6RGY,8DWJ,5-M1VT,Initial Diagnosis,Right,No Glaucoma,,Good


### Crop Image by bounding box


In [ ]:
execution.dataset_paths[0]

In [ ]:
output_dir = execution.working_dir
image_path, cropped_csv = EA.create_cropped_images(execution.dataset_paths[0], ds_bag, output_dir, crop_to_eye=True)

# Execution of ML

In [ ]:
with execution.execute() as exec:
    print("An ML Execution.") 

# Upload results

In [ ]:
# crete asset path
asset_type_name = ""
asset_path = execution.execution_asset_path("Training_Log")
# save assets to asset_path
df.to_csv(asset_path+'filename.csv', index=False)

# upload assets to catalog
execution.upload_execution_outputs(clean_folder=True)

In [ ]:
execution.upload_execution_outputs(clean_folder=True)